In [91]:
import pandas as pd

# Puxando tabelas  da camada silver

In [92]:
df_categorias_s = pd.read_parquet('../silver/data_silver/idv_store_categorias_s.parquet')
df_clientes_s = pd.read_parquet('../silver/data_silver/idv_store_clientes_s.parquet')
df_enderecos_s = pd.read_parquet('../silver/data_silver/idv_store_enderecos_s.parquet')
df_estoque_s = pd.read_parquet('../silver/data_silver/idv_store_estoque_s.parquet')
df_fornecedores_s = pd.read_parquet('../silver/data_silver/idv_store_fornecedores_s.parquet')
df_funcionarios_s = pd.read_parquet('../silver/data_silver/idv_store_funcionarios_s.parquet')
df_itens_pedidos_s = pd.read_parquet('../silver/data_silver/idv_store_itens_pedidos_s.parquet')
df_pagamentos_s = pd.read_parquet('../silver/data_silver/idv_store_pagamentos_s.parquet')
df_pedidos_s = pd.read_parquet('../silver/data_silver/idv_store_pedidos_s.parquet')
df_produtos_s = pd.read_parquet('../silver/data_silver/idv_store_produtos_s.parquet')

# Modelo de Vendas (Fato Vendas)

In [93]:
#Renomeie colunas ambíguas 
df_pedidos_s.rename(columns={'data': 'data_pedido', 'hora': 'hora_pedido'}, inplace=True)
df_clientes_s.rename(columns={'nome_completo': 'nome_cliente', 'cpf': 'cpf_cliente', 'data': 'data_cadastro_cliente', 'hora': 'hora_cadastro_cliente'}, inplace=True)
df_produtos_s.rename(columns={'nome': 'nome_produto', 'descricao': 'descricao_produto', 'data_cadastro': 'data_cadastro_produto'}, inplace=True)
df_categorias_s.rename(columns={'nome': 'nome_categoria', 'descricao': 'descricao_categoria'}, inplace=True)
df_funcionarios_s.rename(columns={'nome_completo': 'nome_funcionario', 'cpf': 'cpf_funcionario'}, inplace=True)
df_pagamentos_s.rename(columns={'data': 'data_pagamento', 'hora': 'hora_pagamento', 'valor': 'valor_pago'}, inplace=True)

In [94]:
# Merge
df_vendas_base = pd.merge(df_itens_pedidos_s, df_pedidos_s, on='id_pedido')
df_vendas_com_cliente = pd.merge(df_vendas_base, df_clientes_s, on='id_cliente')
df_vendas_com_produto = pd.merge(df_vendas_com_cliente, df_produtos_s, on='id_produto')
df_vendas_com_categoria = pd.merge(df_vendas_com_produto, df_categorias_s, on='id_categoria')
df_vendas_com_endereco = pd.merge(df_vendas_com_categoria, df_enderecos_s, on='id_cliente')
df_vendas_com_funcionario = pd.merge(df_vendas_com_endereco, df_funcionarios_s,on='id_funcionario')
df_fato_vendas_bruto = pd.merge(df_vendas_com_funcionario, df_pagamentos_s, on='id_pedido')

In [95]:
# Cópia do DataFrame original
df_fato_vendas_agregada = df_fato_vendas_bruto.copy()

# Conversão para tipos numéricos no DataFrame correto
colunas_numericas = ["preco_unitario_venda", "preco_custo", "quantidade", "valor_pago"]

for col in colunas_numericas:
    df_fato_vendas_agregada[col] = pd.to_numeric(df_fato_vendas_agregada[col], errors="coerce")

# Agregações
df_fato_vendas_agregada["lucro_bruto_item"] = (
    (df_fato_vendas_agregada["preco_unitario_venda"] - df_fato_vendas_agregada["preco_custo"])
    * df_fato_vendas_agregada["quantidade"]
).round(2)

df_fato_vendas_agregada["margem_lucro_percentual"] = df_fato_vendas_agregada.apply(
    lambda row: round((row["lucro_bruto_item"] / row["valor_pago"]) * 100, 2)
    if row["valor_pago"] != 0 else 0, axis=1
)

In [96]:
#Selecione e reordene as colunas desejadas.
colunas_finais = [
    'id_item_pedido',
    'id_pedido',
    'id_produto',
    'id_cliente',
    'id_funcionario',
    'quantidade',
    'preco_unitario_venda',
    'valor_total',
    'preco_custo',
    'preco_venda',
    'lucro_bruto_item',
    'margem_lucro_percentual',
    'metodo_pagamento',
    'nome_produto',
    'nome_categoria',
    'cidade',
    'uf',
    'regiao',
    'status_pedido',
    'data_pedido',
]

df_fato_vendas_final = df_fato_vendas_agregada[colunas_finais]

# Salvando dataframes em parquet

In [97]:
df_fato_vendas_final.to_parquet('../gold/data_gold/idv_store_fato_venda_final.parquet', index=False)